In [1]:
from imdb import IMDb
from collections import defaultdict
import cPickle as pickle  
import io

In [3]:
def get_titles():
    # this file is too big for github, but you cannot download it to your 
    # local computer at this link. I keep my file in the parent directory 
    # of this repository. 
    f = open('../movies.list')
    titles = []
    record = False
    for line in f.readlines():
        if "===========" in line:
            record = True
        if record and line[0] != '"' and line[0].isalnum():
            line_list = line.split('\t')
            line_list2 = line_list[0].split(')')
            line_list3 = line_list2[0].split('(')
            if len(line_list3) >= 2:
                title = line_list3[0]
                year = line_list3[1]
                if "???" not in year:
                    if "{" in line and len(titles) >= 1:
                        del titles[-1]
                    else:
                        titles.append((title, year))
    print "retrieved %d titles" % len(titles)
    return titles

def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            pass
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                return None
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None
        
def get_movies(ia, titles):
    movie_list = []
    for title, year in titles[:100]:
        search_title = title + ' ('+year+')'
        movies = ia.search_movie(search_title)
        movie_title = find_movie(title, year, movies)
        if (movie_title !=  None) and (movie_title['kind'] == unicode('movie')):
            movie = ia.get_movie(movie_title.movieID)
            if movie.data.has_key('genres') and (unicode('Adult') in movie.data['genres']):
                pass
            else:
                movie_list.append(movie)

    print "acquired %d movies" % len(movie_list)
    return movie_list


ia = IMDb()
titles = get_titles()
movies = get_movies(ia, titles)

retrieved 1120479 titles
acquired 23 movies


In [4]:
filename = 'movies.p'
pickle.dump(movies, io.open(filename,'wb'))
movies = pickle.load(io.open(filename,'rb'))
print "Saved %d movies to %s " %(len(movies), filename)

Saved 23 movies to movies.p 
